In [ ]:
include("../src/RadialBasisFunction.jl")
include("../src/DiscreteAdjoint.jl")
include("../src/GradientDescentTools.jl")

using LinearAlgebra, Plots
using .RadialBasisFunction
using .DiscreteAdjoint
using .GradientDescentTools

const dir_for_figures = "../results/figures/"


## Adjoint method applied to RBF approximation

In this notebook, unlike the previous one, the parameter that we want to optimize, $q$ also appear explicitly inside the objecitve function $J$; so in this case we have $J(u,q)$ instead of simply $J(u)$  
The optimization through th eadjoint change in the fact that now we need one more term to compute $\frac{d}{dq} J$

Fixed some paramters, we obtain an approximated field solving the steady-state heat equation

In [ ]:
a, b = 0, 2π
N = 40
X = range(a,b, length=N)
boundary_idx = [1, N]
X_boundary = X[boundary_idx]
internal_idx = 2:(N-1)
X_internal = X[internal_idx]


u(x) = sin.(x)   # Not known in advance
q(x) = -sin.(x)  # Known in advance (param)

boundary_conditions = u(X_boundary)


φ(r) = r .^ 3  # r = |x-x_i| ≥ 0
ddφ(r) = 6*r   #second derivative d²/dx² (because the differential problem deals with u" = d²u/dx²)


n_points_in_stencil = 7


C = global_matrix(X, φ, ddφ, n_points_in_stencil)
u_approximated = solve_RBF(C, X, internal_idx, boundary_idx, boundary_conditions, q)

Now we want to find the field $u_{opt}$ that minimize a cost function $J(u,q)$. We start by defining the cost function:

In [ ]:
# Note that the optimization is carried out only on internal points (in fact boundary poiints are conditioned)...
C_internal = C[internal_idx, internal_idx]

# Defininig J (obj function)
u_target(x) = (x .- a) .* (b .- x)
u_target_num = u_target(X_internal)
c_1, c_2 = 0.4, 0.6
J(u_num, q_num) = c_1*norm(u_num - u_target_num)^2 + c_2*norm(q_num)^2


Set up everything that is required for the optimization with the adjoint method

In [ ]:
# Partial derivatives of J (used to compute the total derivative respect q)
dJdu(u_num, q_num) = 2*c_1*(u_num - u_target_num)
dJdq(u_num, q_num) = 2*c_2*q_num

# Defining the initial values of the constraint with the ones obtained from the RBF approximation
u_RBF = u_approximated
f_RBF = C[internal_idx, boundary_idx] * boundary_conditions
q_RBF = C_internal*u_RBF + f_RBF
constraint = RBFEquation(C_internal, u_RBF, q_RBF, f_RBF)

N_iter::Int64 = 100    # Number of optimization cycles
eval_freq::Int64 = 1   # Frequency of evaluation of the objective function  

To get the sougth parameters $q$ that leads to the desired field $u_{opt}$ we use different iterative gradient methods in the discrete adjoint optimization:
- Barzilai-Borwein with long steps
- Barzilai-Borwein with short steps
- Standard gradient update with fixed steps 

In [ ]:
q_opt_BB_long_num, u_opt_BB_long_num, q_results_BB_long, u_results_BB_long, steps_to_eval = adjoint_opt_BB(dJdu, dJdq, constraint, N_iter, eval_freq, BB_long_step)

q_opt_BB_short_num, u_opt_BB_short_num, q_results_BB_short, u_results_BB_short = adjoint_opt_BB(dJdu, dJdq, constraint, N_iter, eval_freq, BB_short_step)

step_size(∇, n) = 1/norm(∇)
q_opt_grad_fixed_steps_num, u_opt_grad_fixed_steps_num, q_results_grad_fixed_steps, u_results_grad_fixed_steps = adjoint_opt(dJdu, dJdq, constraint, N_iter, eval_freq, step_size)

## Performances

In [ ]:
println("Barzilai-Borwein long steps")
println("Fitness value: $(J(u_opt_BB_long_num, q_opt_BB_long_num))")
println()

println("Barzilai-Borwein short steps")
println("Fitness value: $(J(u_opt_BB_short_num, q_opt_BB_short_num))")
println()

println("Standard gradient fixed steps")
println("Fitness value: $(J(u_opt_grad_fixed_steps_num, q_opt_grad_fixed_steps_num))")

## Plots

Computing the value of $J$ at each step of the optimization...

In [ ]:
J_results_BB_long = [J(u,q) for (u,q) in zip(eachcol(u_results_BB_long), eachcol(q_results_BB_long))]
J_results_BB_short = [J(u,q) for (u,q) in zip(eachcol(u_results_BB_short), eachcol(q_results_BB_short))]
J_results_grad_fixed_steps = [J(u,q) for (u,q) in zip(eachcol(u_results_grad_fixed_steps), eachcol(q_results_grad_fixed_steps))];

... and plotting the results

In [ ]:
p = plot(steps_to_eval, J_results_BB_long, label="Barzilai-Borwein long steps")
plot!(steps_to_eval, J_results_BB_short, label="Barzilai-Borwein short steps")
plot!(steps_to_eval, J_results_grad_fixed_steps, label="Gradient fixed steps")

title!("J(u,q) vs. number of iterations")
xlabel!("Iteration")
ylabel!("J")
plot!(legend=:topright)

# To limit the axis
# ylims!(0, 200)
xlims!(0, 50)

savefig(dir_for_figures*"J(u,q)_vs_iter_gradien_steps_comparison.png")

Note that:

- *Standard gradient fixed step*s does not converge
- *Barzila-Borwein short steps* is the fastest to reach convergence